In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [ ]:
import cv2
import random
from keras.utils import np_utils
import tensorflow as tf

In [ ]:
DATADIR = "/kaggle/input/multiclass-weather-dataset/dataset"
CATEGORIES = ["cloudy","foggy","rainy","shine","sunrise"]

In [ ]:
IMG_SIZE = 224

In [ ]:
#no of images in each folder

# count_labels = dict()

# for category in CATEGORIES:
#     count = 0
#     path = os.path.join(DATADIR,category) 
#     class_num = CATEGORIES.index(category)
#     for img in os.listdir(path):
#         count += 1
#     count_labels[class_num] = count
    
    

# for i in count_labels.keys():
#     print(CATEGORIES[i]," ",count_labels[i])
    
# plt.figure(figsize=(10, 8))
# plt.bar(range(len(count_labels)), list(count_labels.values()), align='center',color = 'black')
# plt.xticks(range(len(count_labels)), list(count_labels.keys()))
# plt.xticks([0,1,2,3,4],CATEGORIES)
# plt.title('No. of images in different labels')
# plt.show()

In [ ]:
# for category in CATEGORIES:
#     path = os.path.join(DATADIR,category) 
#     for img in os.listdir(path):
#         img_array = cv2.imread(os.path.join(path,img))
#         print(img_array)
#         plt.imshow(img_array)
#         plt.show()
#         img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
#         plt.imshow(img_array)
#         plt.show()
#         new_array = cv2.resize(img_array, (50, 50))/255.0
#         print(new_array)
#         plt.imshow(img_array)
#         plt.show()
#         break
#     break

In [ ]:
# Creation of training data 

training_data = []

def create_training_data():
    for category in CATEGORIES:
        path = os.path.join(DATADIR,category) 
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path,img))
                img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))/255.0
                training_data.append([new_array, class_num])  
            except Exception as e:
                pass

            
create_training_data()
print(len(training_data))

In [ ]:
import random
random.shuffle(training_data)

In [ ]:
num_classes = 5
X = []
y = []

for features,label in training_data:
    X.append(features)
    y.append(label)

X = np.array(X)
y = np.array(y)

In [ ]:
# del training_data

In [ ]:
# import tensorflow as tf
# from tensorflow import keras
# import numpy as np

# def build_model(hp):
#     model = keras.Sequential([
#     keras.layers.Conv2D(
#         filters=hp.Int('conv_1_filter', min_value=16, max_value=256, step=16),
#         kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
#         activation='relu',
#         input_shape=(IMG_SIZE,IMG_SIZE,3)
#     ),
#     keras.layers.Conv2D(
#     filters=hp.Int('conv_2_filter', min_value=16, max_value=256, step=16),
#     kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
#     activation='relu'
#     ),
#     keras.layers.Flatten(),
#     keras.layers.Dense(
#         units=hp.Int('dense_1_units', min_value=16, max_value=512, step=16),
#         activation='relu'
#     ),
#     keras.layers.Dense(5, activation='softmax')
#     ])
#     model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
#               loss='sparse_categorical_crossentropy',
#               metrics=['accuracy'])
#     return model

In [ ]:
# from kerastuner import RandomSearch
# from kerastuner.engine.hyperparameters import HyperParameters

In [ ]:
# tuner_search=RandomSearch(build_model,
#                           objective='val_accuracy',
#                           max_trials=3)

In [ ]:
# tuner_search.search(X,y,epochs=3)

In [ ]:
# VGG 16 ..................................................................................................................

In [ ]:
# Training model using VGG-16 

from keras.layers import  Flatten, Dense, Dropout
from keras.applications import VGG16
from keras.models import Model
from keras import optimizers
from keras.optimizers import Adam
from keras.layers import Dense, GlobalAveragePooling2D
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization, AveragePooling2D, GlobalAveragePooling2D

vgg16_model = VGG16(weights = 'imagenet', include_top = False,input_shape=(224,224,3))
x = vgg16_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

In [ ]:
num_classes = 5

In [ ]:
predictions = Dense(num_classes, activation = 'softmax')(x)
model2 = Model(vgg16_model.input,predictions)
for layer in vgg16_model.layers:
    layer.trainable = False
optimizer = Adam(lr=0.0002)
model2.compile(loss='sparse_categorical_crossentropy', optimizer = optimizer, metrics=['accuracy'])

In [ ]:
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator
training_datagen = ImageDataGenerator(rotation_range=40, 
                        width_shift_range=0.2, 
                        height_shift_range=0.2, 
                        zoom_range=0.2, 
                        horizontal_flip=True, 
                        vertical_flip=True,
                        shear_range=0.2) 

train_generator = training_datagen.flow(X, y,batch_size=64)
training_datagen.fit(X)

In [ ]:
# del X
# del y

In [ ]:
DATADIR = "/kaggle/input/multiclass-weather-dataset/dataset/alien_test"
CATEGORIES = ["Cloud","foggy","rain","shine","sunrise"]

In [ ]:
# Creation of testing data

testing_data = []
def create_testing_data():
    for img in os.listdir(DATADIR):
        try:
            category = img.split("_")[0]
            class_num = CATEGORIES.index(category)
            img_array = cv2.imread(os.path.join(DATADIR,img))
            img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
            new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))/255.0
            testing_data.append([new_array, class_num])  
        except Exception as e:
            pass

create_testing_data()
print(len(testing_data))

In [ ]:
import random
random.shuffle(testing_data)

In [ ]:
X_test = []
y_test = []

for features,label in testing_data:
    X_test.append(features)
    y_test.append(label)
    
X_test = np.array(X_test)
y_test = np.array(y_test)

In [ ]:
# del testing_data

In [ ]:
from keras import callbacks
filepath="Best1.hdf5"
checkpoint = callbacks.ModelCheckpoint(filepath, monitor='val_loss',save_best_only=True, mode='min',verbose=1)
callbacks_list = [checkpoint]

history = model2.fit_generator(train_generator,epochs=20,validation_data=(X_test, y_test),callbacks=callbacks_list)

In [ ]:
y_pred = model2.predict(X_test)

array = model2.predict(X_test, batch_size=1, verbose=1)
y_pred = np.argmax(array, axis=1)

from sklearn.metrics import confusion_matrix,accuracy_score
acc = accuracy_score(y_test,y_pred)
acc = acc*100
cf = confusion_matrix(y_test, y_pred)
print(cf)
print("Accuracy is : ",round(acc,2),"%")

In [ ]:
model2.load_weights("Best1.hdf5")
score = model2.evaluate(X_test, y_test ,verbose=1)
print('Test Loss is :', score[0])
print('Test accuracy is :', score[1])

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(acc))
plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('No of epochs')
plt.ylabel('Accuracy')
plt.legend(loc=0)
plt.figure()
plt.show()

In [ ]:
print(y_test)
print(y_pred)

In [ ]:
# plt.imshow(X_test[0])
# plt.show()
# print(y_pred[0])
# pritn(y_test[0])

In [ ]:
# ResNet ...............................................................................................................

In [ ]:
from keras.models import load_model
from sklearn.datasets import load_files
from keras.utils import np_utils
from glob import glob
from keras import applications
from keras.preprocessing.image import ImageDataGenerator 
from keras import optimizers
from keras.models import Sequential,Model,load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,GlobalAveragePooling2D
from keras.callbacks import TensorBoard,ReduceLROnPlateau,ModelCheckpoint

base_model = applications.resnet50.ResNet50(weights= None, include_top=False, input_shape= (IMG_SIZE,IMG_SIZE,3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.7)(x)
predictions = Dense(num_classes, activation= 'softmax')(x)
model = Model(inputs = base_model.input, outputs = predictions)
from keras.optimizers import SGD, Adam
adam = Adam(lr=0.0001)
model.compile(optimizer= adam, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X, y, epochs = 30, batch_size = 8)

In [ ]:
y_pred = model.predict(X_test)
array = model2.predict(X_test, batch_size=1, verbose=1)
y_pred = np.argmax(array, axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score
acc = accuracy_score(y_test,y_pred)
acc = acc*100
cf = confusion_matrix(y_test, y_pred)
print(cf)
print("Accuracy is : ",round(acc,2),"%")